<a href="https://colab.research.google.com/github/Bilal-Subhopoto/DataEngineering-Bilal/blob/main/GoogleFBscraper2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tldextract 

In [ ]:
!pip install facebook_scraper

In [ ]:
!pip install redis

In [11]:
from googlesearch import search
import tldextract
import time
import random
import redis
from facebook_scraper import get_posts
masterDB = redis.Redis(host='43.251.253.107', port=1500, db=0)
DB1 = redis.Redis(host='43.251.253.107', port=1500, db=1)
DB2 = redis.Redis(host='43.251.253.107', port=1500, db=2)
import pymongo
myclient = pymongo.MongoClient("mongodb://43.251.253.107:1600/")
mydb = myclient['GoogleFBscraper2-Bilal']
mycol = mydb['Allposts']

In [5]:
# read sets' keys from redis masterDB and decode
sitenames_list = []
for keyb in masterDB.scan_iter():
  print(keyb)
  keyS = keyb.decode("utf-8")
  sitenames_list.append(keyS)# read all the keys from redis into python list
  # masterDB.spop(key1)# loop over the above list and delete each entry from redis
print(sitenames_list)#simple strings inside list, NOT byte

b'Time'
b'Telegraph'
b'Business Insider'
b'Boston.com'
b'Boston Globe'
b'Fox News Digital Network'
b'New York Daily News'
b'CBS News'
b'Huffington Post'
b'Bleacher Report'
b'MSN News'
b'Detroit Free Press'
b'Los Angeles Times'
b'Examiner'
b'Atlantic'
b'Slate'
b'Chron'
b'Salon'
b'Mic'
b'BBC'
b'CNET'
b'New York Post'
b'Washington Post'
b'Vice'
b'Yahoo-ABC News Network'
b'NPR'
b'New York Times'
b'U.S. News'
b'Upworthy'
b'AL.com'
b'Chicago Tribune'
b'NBC News'
b'Mashable'
b'TechCrunch'
b'Engadget'
b'USAToday'
b'SFGate'
b'TheBlaze'
b'BuzzFeed'
b'Daily Mail'
b'Independent'
b'Guardian'
b'Dallas Morning News'
b'MLive'
b'Mirror Online'
b'Daily Beast'
b'Elite Daily'
b'CNN'
['Time', 'Telegraph', 'Business Insider', 'Boston.com', 'Boston Globe', 'Fox News Digital Network', 'New York Daily News', 'CBS News', 'Huffington Post', 'Bleacher Report', 'MSN News', 'Detroit Free Press', 'Los Angeles Times', 'Examiner', 'Atlantic', 'Slate', 'Chron', 'Salon', 'Mic', 'BBC', 'CNET', 'New York Post', 'Washingto

In [6]:
# google-search each sitename from the above list
urls = []
for ttls in sitenames_list:
  
  try:
    resp=search("facebook.com "+ttls, num=2)   
    time.sleep(random.randint(5,12))
    for link in resp:
      childFLD=tldextract.extract(link)
      childFLD=childFLD.domain
      if childFLD=="facebook":
        try:
          print(link)
          pageNames = link.split('/')[3].strip()
          # pageNames = link.split('/')[2:3]use either
          print(pageNames)
          DB1.sadd(pageNames,'sitename') # insert the extracted unique fb page name into redis DB-1
          masterDB.spop(ttls) # in the for loop of list from redis after finding unique page name on google delete that particular key of sitename from DB-0
        except Exception as e:
          print(e)
          break
        break
  except Exception as e:
    print(e)
    break
# make sure that the key(s) is deleted from DB-0 only when a succesful unique Fb page name is found.
# make sure when google blocks us the program should enddddd
# this concept is state management in DB-0 

https://www.facebook.com/time/
time
https://www.facebook.com/TELEGRAPH.CO.UK/
TELEGRAPH.CO.UK
https://www.facebook.com/businessinsider/
businessinsider
https://www.facebook.com/boston/
boston
https://www.facebook.com/globe/
globe
https://www.facebook.com/FoxNews
FoxNews
https://www.facebook.com/NYDailyNews/
NYDailyNews
https://www.facebook.com/CBSNews/
CBSNews
https://www.facebook.com/HuffPost/
HuffPost
https://www.facebook.com/bleacherreport/
bleacherreport
https://www.facebook.com/msn/
msn
https://www.facebook.com/detroitfreepress/
detroitfreepress
https://www.facebook.com/latimes/
latimes
https://www.facebook.com/examiner.net/
examiner.net
https://www.facebook.com/TheAtlantic/
TheAtlantic
https://www.facebook.com/Slate/
Slate
https://www.facebook.com/chroncom/
chroncom
HTTP Error 429: Too Many Requests


In [7]:
# read sets' keys from redis DB1 and decode
for keyb1 in DB1.scan_iter():
  print(keyb1)
  keyS1 = keyb1.decode("utf-8")
  urls.append(keyS1)# read all the keys from redis into python list
  # masterDB.spop(key1)# loop over the above list and delete each entry from redis
print(urls)#simple strings inside list, NOT byte from DB1

b'msn'
b'Vox'
b'bleacherreport'
b'chroncom'
b'boston'
b'latimes'
b'examiner.net'
b'TELEGRAPH.CO.UK'
b'detroitfreepress'
b'NYDailyNews'
b'HuffPost'
b'Gawker'
b'NJ.com'
b'Slate'
b'businessinsider'
b'TheAtlantic'
b'CBSNews'
b'FoxNews'
b'time'
b'globe'
['msn', 'Vox', 'bleacherreport', 'chroncom', 'boston', 'latimes', 'examiner.net', 'TELEGRAPH.CO.UK', 'detroitfreepress', 'NYDailyNews', 'HuffPost', 'Gawker', 'NJ.com', 'Slate', 'businessinsider', 'TheAtlantic', 'CBSNews', 'FoxNews', 'time', 'globe']


In [13]:
# fb urls to extract posts data and upload on DB-1
fields = ('comments','link','likes','images','shared_time','text','post_id','reactions','shares','user_id','time','timestamp','time')
# fbPages = ('ABCNews','cnn','NBCNews','mlive','TheIndependentOnline','thedailybeast','WashingtonExaminer','Gawker','Upworthy','chroncom','Slate','bleacherreport','EliteDaily','washingtonpost','DailyMail','nytimes','BuzzFeed','usatoday','CBSNews','HuffPost')
for fbp in urls:
  try:
    for post in get_posts(fbp, pages= 3,cookies= 'cookies.txt'):
      collection = {'pageName':fbp}    
      for items in fields:
        collection[items] = (post[items])
        # DB2.hset(fbp,items,post[items])
      x = mycol.insert_one(collection) 
  except Exception as e:
    print(e)